In [1]:
import pandas as pd
import re

In [2]:
Data = pd.read_csv("JobDataTranslated.csv")
keyWords = pd.read_csv("words_set_long.csv")
keyWords_short = pd.read_csv("words_set_short.csv")
Data=Data[:-20]

In [3]:
# just a thing to clean input
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

#Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())

In [4]:
kWords=[keyWords_short['0'][i] for i in range(len(keyWords_short))]
#kWords=[keyWords['0'][i] for i in range(len(keyWords))]

In [5]:
Data['Cleaned Job Description']

0      you know better than anyone how to bind other ...
1      do you want to get paid to drive around your c...
2      document controller do you ensure that all dig...
3      our growing retail sector is looking for a ski...
4      working from home at conduent, start your appl...
                             ...                        
975    company description zerogas is a young company...
976    for a medical laboratory in utrecht, we are im...
977    permanent acting gp / hidha at quin dokters ne...
978    vacancy b2b online marketer hello visitor, bli...
979    the social support act is a department within ...
Name: Cleaned Job Description, Length: 980, dtype: object

In [6]:

""" this thing allows us to find all words start indexes"""
def find_all(text, sub):
    start = 0
    while True:
        start = text.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

"""all words start indexes for list of words"""
def find_all_in_list(text, subs: list):
    l=[]
    for i in subs:
        l.append(list(find_all(text,i)))
    m = [l[j][k] for j in range(len(l)) for k in range(len(l[j]))]
    return m


"gives a sentence for an index of word inside"
def get_sentence(num, text ):
    l= find_all_in_list(text, [',','.','!',';','!',':','/'])
    l.sort()
    if (num <l[0]):
        return text[0:l[0]]
    elif (num>l[-1]):
        return text[l[-1]+1:]
    
    x=[i for i in range(len(l)) if l[i]<num and l[i+1]>num]
    return (text[l[x[0]]+1:l[x[0]+1]])


In [7]:
l = []
for d in Data['Cleaned Job Description'] :
    for i in kWords:
        if d.find(i)!=-1:
            print(d.find(i),d.find(i)+len(i), get_sentence(d.find(i),d) ,'hui',i)
            l.append((get_sentence(d.find(i),d) ,{"entities": [(d.find(i),d.find(i)+len(i), "SKILL")]}))

406 422  then a role as a sales consultant is perfect for you hui sales consultant
1085 1101  maintenance & asset management hui asset management
1708 1731  a college degree from a commercial or technical related study at least 1 year of experience in sales and preferably in job placement a lot of ambition to grow in your profession a representative attitude strongly developed communication skills and a lot of persuasiveness and a valid driver license b hui developed communication
1497 1511  a college degree from a commercial or technical related study at least 1 year of experience in sales and preferably in job placement a lot of ambition to grow in your profession a representative attitude strongly developed communication skills and a lot of persuasiveness and a valid driver license b hui college degree
1718 1738  a college degree from a commercial or technical related study at least 1 year of experience in sales and preferably in job placement a lot of ambition to grow in your profe

In [8]:
l[30:40]

[(' you are partly responsible for the implementation and you will mainly perform system management activities',
  {'entities': [(875, 896, 'SKILL')]}),
 (' what do you need? what do we expect from you? experience in installation technology',
  {'entities': [(1233, 1256, 'SKILL')]}),
 ('what are you going to do? do you see opportunities in technology? and do you want to be trained as an experienced service technician? then i have a great opportunity for you here',
  {'entities': [(114, 132, 'SKILL')]}),
 (' are you a starter in technology and are you ready to take steps? as a starter you will be trained within the organization as a service technician in heating technology',
  {'entities': [(646, 664, 'SKILL')]}),
 ('what are you going to do? do you see opportunities in technology? and do you want to be trained as an experienced service technician? then i have a great opportunity for you here',
  {'entities': [(102, 132, 'SKILL')]}),
 (' as an apprentice service technician you work in a

In [9]:
len(l)

7187

In [10]:
import json
with open("training_set_1.txt", "w") as fp:
    json.dump(l, fp)

    
# that's how you load this thing, the format will be mimimally diffreent so either fix it or just merge 2 notebooks
#with open("training_set_1.txt", "r") as fp:
    #b = json.load(fp)